In [69]:
import json
import os

In [2]:
from bs4 import BeautifulSoup
import cloudscraper
import geopandas as gpd
import pandas as pd

In [3]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [5]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [6]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [7]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [8]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Family Dollar Data

In [9]:
scraper = cloudscraper.create_scraper()

In [93]:
store_list = []
url_base = "https://locations.familydollar.com"
for i in tqdm(range(len(states_df)), desc="Parsing States"):
    # Get State code
    state_store_list = []
    state_code = states_df.iloc[i]["STUSPS"].lower()

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    # Get all locations in a state
    state_url = os.path.join(url_base, state_code)
    r_state = scraper.get(state_url, allow_redirects=False)

    if r_state.status_code != 200:
        print(states_df.iloc[i]["NAME"])
        continue

    soup_state = BeautifulSoup(r_state.text, "html.parser")
    city_as = soup_state.find_all(
        "a",
        {
            "class": "Link group flex items-center gap-2 text-base font-normal !no-underline"
        },
    )

    for city_a in tqdm(city_as, desc=f"Parsing cities in {state_code.upper()}"):
        city_url = os.path.join(url_base, city_a.attrs["href"])
        r_city = scraper.get(city_url)
        soup_city = BeautifulSoup(r_city.text, "html.parser")
        store_as = soup_city.find_all(
            "a", {"class": "Link text-green1 pb-4 text-lg font-bold hover:underline"}
        )
        for store_a in store_as:
            store_url = os.path.join(url_base, store_a.attrs["href"][1:])
            r_store = scraper.get(store_url)
            soup_store = BeautifulSoup(r_store.text, "html.parser")
            try:
                store_dict = json.loads(
                    soup_store.find("script", {"type": "application/ld+json"}).text
                )
                geo = store_dict["@graph"][1]["geo"]
                store_dict = {
                    "STUSPS": state_code.upper(),
                    "geometry": Point(geo["longitude"], geo["latitude"]),
                }
            except Exception as e:
                print(r_store.url, e)
                continue
            state_store_list.append(store_dict)
            store_list.append(store_dict)

    if state_store_list:
        state_family_dollar_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
        state_family_dollar_gdf.to_file(f"data/states/{state_code}.gpkg")

Parsing States:   0%|          | 0/56 [00:00<?, ?it/s]

Parsing cities in NC:   0%|          | 0/230 [00:00<?, ?it/s]

https://locations.familydollar.com/nc/camden/115-hwy-158 'longitude'
https://locations.familydollar.com/nc/charlotte/7524-s-tryon-street 'longitude'
https://locations.familydollar.com/nc/gastonia/2206-union-rd 'longitude'
https://locations.familydollar.com/nc/greenville/250a-easy-st 'longitude'
https://locations.familydollar.com/nc/monroe/106-e-sunset-dr 'longitude'
https://locations.familydollar.com/nc/monroe/807-e-roosevelt-blvd 'longitude'


Parsing cities in OK:   0%|          | 0/107 [00:00<?, ?it/s]

Parsing cities in VA:   0%|          | 0/131 [00:00<?, ?it/s]

...

https://locations.familydollar.com/ar/little-rock/2408-dr-martin-luther-king-jr 'longitude'


Parsing cities in IN:   0%|          | 0/99 [00:00<?, ?it/s]

https://locations.familydollar.com/in/elkhart/2301-west-franklin-st 'longitude'
https://locations.familydollar.com/in/indianapolis/3021-w-washington-st 'longitude'
https://locations.familydollar.com/in/nashville/51-chestnut-st 'longitude'
https://locations.familydollar.com/in/valparaiso/390-w-us-highway-6 'longitude'


Parsing cities in MN:   0%|          | 0/43 [00:00<?, ?it/s]

Parsing cities in CT:   0%|          | 0/31 [00:00<?, ?it/s]

#### If the above cells breaks just keep running until complete. Then run the line below

In [94]:
family_dollars_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        family_dollars_gdf = gpd.read_file(constructed_path)
        family_dollars_gdfs.append(family_dollars_gdf)

In [95]:
family_dollar_gdf = gpd.GeoDataFrame(pd.concat(family_dollars_gdfs, ignore_index=True))

In [101]:
added_family_dollars = [
    {"STUSPS": "IN", "geometry": Point(-86.00190044047244, 41.6702614070786)},
    {"STUSPS": "IN", "geometry": Point(-86.23774448460554, 39.20065518475156)},
    {"STUSPS": "IN", "geometry": Point(-86.21062951349928, 39.76444922942507)},
    {"STUSPS": "IN", "geometry": Point(-87.14129259833024, 41.549362429073526)},
    {"STUSPS": "OH", "geometry": Point(-81.5485640536069, 41.55953042310425)},
    {"STUSPS": "OH", "geometry": Point(-84.1918123441257, 39.434046064951964)},
    {"STUSPS": "OH", "geometry": Point(-81.45643240189779, 40.5041574677327)},
    {"STUSPS": "OH", "geometry": Point(-80.6127432306313, 40.45186429734367)},
    {"STUSPS": "KY", "geometry": Point(-83.67636775766854, 37.47705711407493)},
    {"STUSPS": "KY", "geometry": Point(-83.78725459631947, 37.21538858377342)},
    {"STUSPS": "KY", "geometry": Point(-84.48301721535492, 38.06024457121491)},
    {"STUSPS": "KY", "geometry": Point(-82.75365220000161, 37.55744285259612)},
    {"STUSPS": "KY", "geometry": Point(-86.69536448281158, 37.211455327657106)},
    {"STUSPS": "KY", "geometry": Point(-87.10447757293281, 37.75792045398469)},
    {"STUSPS": "KY", "geometry": Point(-82.5438040711658, 37.3727348581624)},
    {"STUSPS": "KY", "geometry": Point(-86.88962764231462, 36.84662503559783)},
    {"STUSPS": "KY", "geometry": Point(-83.12494447117369, 38.09106698276062)},
    {"STUSPS": "RI", "geometry": Point(-71.39241224038442, 41.88241772065835)},
    {"STUSPS": "AZ", "geometry": Point(-109.95374082884541, 34.142421280502255)},
    {"STUSPS": "SC", "geometry": Point(-82.18887927117117, 34.521087900811466)},
    {"STUSPS": "SC", "geometry": Point(-81.84529138649157, 33.517750848482144)},
    {"STUSPS": "SC", "geometry": Point(-80.90392268466597, 33.47924457682706)},
    {"STUSPS": "IA", "geometry": Point(-93.6249386288345, 41.54871598530128)},
    {"STUSPS": "IL", "geometry": Point(-87.58608869824508, 41.7449678480658)},
    {"STUSPS": "IL", "geometry": Point(-87.67934023068273, 41.82971710277755)},
    {"STUSPS": "IL", "geometry": Point(-87.81135978830129, 41.57968688508415)},
    {"STUSPS": "NY", "geometry": Point(-106.38595267481378, 42.85138213608213)},
    {"STUSPS": "NY", "geometry": Point(-77.65958822706611, 42.32106253915318)},
    {"STUSPS": "NY", "geometry": Point(-73.85712704408405, 40.671316945262106)},
    {"STUSPS": "NY", "geometry": Point(-79.03671259999955, 43.17150258484927)},
    {"STUSPS": "NY", "geometry": Point(-73.82584418842217, 41.74547701706317)},
    {"STUSPS": "NY", "geometry": Point(-74.03927697117894, 41.122978002814364)},
    {"STUSPS": "WY", "geometry": Point(-106.38595267481378, 42.85138213608213)},
    {"STUSPS": "CO", "geometry": Point(-104.75609099999886, 38.78780140706765)},
    {"STUSPS": "UT", "geometry": Point(-111.89281904236756, 40.87250496464274)},
    {"STUSPS": "TN", "geometry": Point(-89.79147577287443, 35.175991923497996)},
    {"STUSPS": "MO", "geometry": Point(-90.35410036944737, 38.78539779827846)},
    {"STUSPS": "MO", "geometry": Point(-94.43752244053958, 39.080590693979445)},
    {"STUSPS": "PA", "geometry": Point(-79.56613769999605, 40.58177898428519)},
    {"STUSPS": "PA", "geometry": Point(-76.84114237117359, 40.27124826084859)},
    {"STUSPS": "PA", "geometry": Point(-76.25338674773332, 40.68101891444123)},
    {"STUSPS": "PA", "geometry": Point(-75.27080464586415, 41.170411600111564)},
    {"STUSPS": "PA", "geometry": Point(-75.23845541173361, 39.945286709902454)},
    {"STUSPS": "GA", "geometry": Point(-83.3109476558399, 34.57383458073483)},
    {"STUSPS": "GA", "geometry": Point(-85.31258484414224, 34.916039015915565)},
    {"STUSPS": "GA", "geometry": Point(-84.55626997306071, 33.93396895987306)},
    {"STUSPS": "GA", "geometry": Point(-83.71716571351006, 32.82740503017334)},
    {"STUSPS": "GA", "geometry": Point(-84.3384700423292, 33.48572622596157)},
    {"STUSPS": "GA", "geometry": Point(-84.76471778829722, 34.82565696021618)},
    {"STUSPS": "AL", "geometry": Point(-87.48846695580735, 31.02149008583219)},
    {"STUSPS": "AL", "geometry": Point(-86.96486711713946, 34.73083572184487)},
    {"STUSPS": "CA", "geometry": Point(-115.51821604229917, 32.97828610060511)},
    {"STUSPS": "CA", "geometry": Point(-115.96689718469418, 33.280298699821834)},
    {"STUSPS": "TX", "geometry": Point(-95.70572041349213, 32.45029662679289)},
    {"STUSPS": "TX", "geometry": Point(-95.0854777711656, 30.3412968516855)},
    {"STUSPS": "TX", "geometry": Point(-96.5456265541155, 33.75434311718546)},
    {"STUSPS": "TX", "geometry": Point(-97.09192001350945, 33.213646031430365)},
    {"STUSPS": "TX", "geometry": Point(-98.16568484415778, 26.337955216950707)},
    {"STUSPS": "TX", "geometry": Point(-106.24612419820042, 31.79637752835674)},
    {"STUSPS": "TX", "geometry": Point(-95.18650398072744, 29.78737899109057)},
    {"STUSPS": "TX", "geometry": Point(-95.49153077296404, 29.64393734094431)},
    {"STUSPS": "TX", "geometry": Point(-95.54463311349907, 30.72429286567833)},
    {"STUSPS": "TX", "geometry": Point(-95.5573693711604, 30.724130234361233)},
    {"STUSPS": "TX", "geometry": Point(-99.4761263711583, 27.470824193415428)},
    {"STUSPS": "TX", "geometry": Point(-94.66231567116841, 31.596557677328857)},
    {"STUSPS": "TX", "geometry": Point(-94.67658595768103, 31.701712248174328)},
    {"STUSPS": "TX", "geometry": Point(-94.68528101351347, 33.199354468237544)},
    {"STUSPS": "TX", "geometry": Point(-102.39437547116668, 31.85192047724366)},
    {"STUSPS": "NC", "geometry": Point(-76.16896464415308, 36.32973481808488)},
    {"STUSPS": "NC", "geometry": Point(-80.92643142702566, 35.15033017961243)},
    {"STUSPS": "NC", "geometry": Point(-81.1672044270553, 35.23243878322777)},
    {"STUSPS": "NC", "geometry": Point(-80.54793858647086, 34.97126188794333)},
    {"STUSPS": "NC", "geometry": Point(-80.51663346771046, 34.97476512684653)},
    {"STUSPS": "NC", "geometry": Point(-76.16896464415308, 36.32973481808488)},
    {"STUSPS": "VA", "geometry": Point(-77.9842807576537, 39.15191820050503)},
    {"STUSPS": "VA", "geometry": Point(-77.08918856336129, 37.034590040145765)},
    {"STUSPS": "WV", "geometry": Point(-81.07755668297234, 38.45956868528548)},
    {"STUSPS": "WV", "geometry": Point(-80.6524573034488, 38.32429655482337)},
    {"STUSPS": "WV", "geometry": Point(-81.34408231531711, 37.778783760685016)},
    {"STUSPS": "WV", "geometry": Point(-80.81851705415528, 38.93856088625672)},
    {"STUSPS": "WV", "geometry": Point(-82.12639245765179, 38.0303844199048)},
    {"STUSPS": "WV", "geometry": Point(-81.87140125412155, 37.73017720280165)},
    {"STUSPS": "WV", "geometry": Point(-77.95699731530108, 39.478028509437614)},
    {"STUSPS": "WV", "geometry": Point(-78.97657594235173, 39.051909985001146)},
    {"STUSPS": "LA", "geometry": Point(-93.76395047297446, 32.47058586484867)},
    {"STUSPS": "MI", "geometry": Point(-84.15183878243023, 45.00671692407967)},
    {"STUSPS": "MI", "geometry": Point(-85.85013417117291, 43.90248148309661)},
    {"STUSPS": "MI", "geometry": Point(-83.67150025587023, 42.96401501629172)},
    {"STUSPS": "MI", "geometry": Point(-87.44101221533134, 46.28105046266114)},
    {"STUSPS": "MI", "geometry": Point(-82.88318031315572, 42.58841466960552)},
    {"STUSPS": "MA", "geometry": Point(-71.18626068645446, 42.72541329926639)},
    {"STUSPS": "FL", "geometry": Point(-81.59554239818982, 30.205655131163144)},
    {"STUSPS": "FL", "geometry": Point(-81.33917658649055, 28.320750565121063)},
    {"STUSPS": "FL", "geometry": Point(-82.12284721720108, 28.003031236310946)},
    {"STUSPS": "FL", "geometry": Point(-87.48824102513714, 30.99405560143657)},
    {"STUSPS": "FL", "geometry": Point(-80.0906626441763, 26.651981205561654)},
    {"STUSPS": "NE", "geometry": Point(-95.94812667305307, 41.27289078591315)},
    {"STUSPS": "NM", "geometry": Point(-106.53191027299363, 35.09541775426459)},
    {"STUSPS": "NM", "geometry": Point(-106.46888784413005, 35.38421420277683)},
    {"STUSPS": "NM", "geometry": Point(-104.33553810373544, 33.11359236579112)},
    {"STUSPS": "NM", "geometry": Point(-104.5237144152589, 33.38200174496823)},
    {"STUSPS": "SD", "geometry": Point(-101.24256438653022, 45.00752311992521)},
]

In [108]:
manually_added_fd_gdf = gpd.GeoDataFrame(added_family_dollars, crs=4326)

In [109]:
family_dollar_with_manual_gdf = gpd.GeoDataFrame(
    pd.concat([family_dollar_gdf, manually_added_fd_gdf], ignore_index=True)
)

In [110]:
family_dollar_with_manual_gdf = family_dollar_with_manual_gdf.to_crs(9311)
family_dollar_with_manual_gdf.to_file(f"data/family_dollars.gpkg")

## Combine With States

In [116]:
family_dollar_state_counts_df = pd.DataFrame(
    family_dollar_with_manual_gdf.groupby("STUSPS").size(), columns=["FAMILY_DOLLARS"]
).reset_index()

In [117]:
family_dollar_per_states_gdf = states_with_population_df.merge(
    family_dollar_state_counts_df, on="STUSPS", how="left"
)

In [119]:
family_dollar_per_states_gdf = family_dollar_per_states_gdf.fillna(0)
family_dollar_per_states_gdf["FAMILY_DOLLARS"] = family_dollar_per_states_gdf[
    "FAMILY_DOLLARS"
].astype(int)

In [120]:
family_dollar_per_states_gdf["per_1000"] = family_dollar_per_states_gdf[
    "FAMILY_DOLLARS"
] / (family_dollar_per_states_gdf["POPULATION"] / 1000)
family_dollar_per_states_gdf["per_10k"] = family_dollar_per_states_gdf[
    "FAMILY_DOLLARS"
] / (family_dollar_per_states_gdf["POPULATION"] / 10_000)
family_dollar_per_states_gdf["per_100k"] = family_dollar_per_states_gdf[
    "FAMILY_DOLLARS"
] / (family_dollar_per_states_gdf["POPULATION"] / 100000)
family_dollar_per_states_gdf["per_500k"] = family_dollar_per_states_gdf[
    "FAMILY_DOLLARS"
] / (family_dollar_per_states_gdf["POPULATION"] / 500_000)
family_dollar_per_states_gdf["per_1m"] = family_dollar_per_states_gdf[
    "FAMILY_DOLLARS"
] / (family_dollar_per_states_gdf["POPULATION"] / 1_000_000)

In [121]:
family_dollar_per_states_gdf = family_dollar_per_states_gdf.dropna()

In [122]:
family_dollar_per_states_gdf = family_dollar_per_states_gdf.to_crs(9311)
family_dollar_per_states_gdf.to_file(f"data/family_dollar_per_state.gpkg")